In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, sys
import numpy as np
import h5netcdf

import xarray as xr
import xrft
import pandas as pd

import dask.array as da
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
#
# Initialisation d'un cluster de 32 coeurs
# cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4)
client = Client(dashboard_address = '137.129.155.67:8787' , host = '137.129.155.67:8788')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:8787/status,
Dashboard: http://137.129.155.67:8787/status,Workers: 4
Total threads: 12,Total memory: 78.61 GiB
Status: running,Using processes: True
Comm: tcp://137.129.155.67:8788,Workers: 4
Dashboard: http://137.129.155.67:8787/status,Total threads: 12
Started: Just now,Total memory: 78.61 GiB
Comm: tcp://137.129.155.67:49131,Total threads: 3
Dashboard: http://137.129.155.67:43045/status,Memory: 19.65 GiB
Nanny: tcp://137.129.155.67:41523,


In [2]:
indir = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/TF2D/TCWV/'
var = 'tcwv'


ds = xr.open_mfdataset(indir+'TF2D_tcwv_ERA5_3H_1992.nc', parallel=True, engine='h5netcdf', invalid_netcdf=True,chunks = 'auto')
ds

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f6599064dd0>, <function open_dataset at 0x7f6592bf0c20>, ['/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/TF2D/TCWV/TF2D_tcwv_ERA5_3H_1992.nc'], (<class 'dict'>, [['engine', 'h5netcdf'], ['chunks', 'auto'], ['invalid_netcdf', True]])))
kwargs:    {}
Exception: OSError('Unable to open file (bad object header version number)')



OSError: Unable to open file (bad object header version number)

In [ ]:
tcwv = xrft.ifft(tcwvhat,
                dim=['freq_time','freq_longitude'], true_phase=False, true_amplitude=True)

tcwvhat = tcwvhat.to_dataset(name = 'TF_tcwv')
tcwvhat['time'] = ds.time
# tcwvhat.to_netcdf('TF2D_tcwv_ERA5_3H_'+str(y)+'.nc', unlimited_dims={'time':True},  engine='h5netcdf', invalid_netcdf=True)

#     tcwv['time'] = ds.time
#     tcwv = tcwv.sel(time = slice(str(y)+"-01-01",str(y)+"-12-31"))
#     tcwv = tcwv.to_dataset(name = 'inverse_TF_tcwv')
# #     tcwv.to_netcdf(outdir_TF + 'TF2D_tcwv_ERA5_3H_'+str(y)+'.nc', unlimited_dims={'time':True})
    
    
    

/home/durandy/miniconda3/envs/croco/lib/python3.7/site-packages/xrft/xrft.py:556: FutureWarning: Default idft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)
/home/durandy/miniconda3/envs/croco/lib/python3.7/site-packages/h5netcdf/core.py:478: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  h5ds.dims.create_scale(h5ds, scale_name)


In [13]:
def Rossby_filter(varhat, periode, wav_min, wav_max):
    radius = 6371000
    
    for l in varhat.latitude.values :
        radian = l*np.pi/180
        _varhat = varhat.sel(latitude = l)
        
        wavelength_max = (np.pi*2*6371000/wav_min)/111.11*np.cos(radian) #!!!! *np.cos() ou /np.cos ??
        wavelength_min = (np.pi*2*6371000/wav_max)/111.11*np.cos(radian)

        _varhat = xr.where(varhat.freq_longitude < -wavelength_min,  0., _varhat) 
        _varhat = xr.where(varhat.freq_longitude > wavelength_max,  0., _varhat)
    
    
    freq_filter = 1 / periode
    varhat = xr.where(varhat.freq_time < -freq_filter,  0., varhat )
    varhat = xr.where(varhat.freq_time > freq_filter,  0., varhat )
    

    _varhat = _varhat.transpose("freq_time",...)
#         print(_varhat.swap_dims({"freq_longitude": "freq_time"}))
    tcwvhat.loc[dict(latitude = str(l))] = _varhat


    return varhat

In [14]:
tcwvhat_Rossby_Filt = Rossby_filter(tcwvhat, 6, -14, -2)
# tcwv = xrft.ifft(tcwv_Ross_Filt['TF_tcwv'],
#                     dim=['freq_time','freq_longitude'], true_phase=False, true_amplitude=True)
tcwvhat_Rossby_Filt

<xarray.Dataset>
Dimensions:         (freq_time: 4840, latitude: 81, freq_longitude: 1440, time: 4840)
Coordinates:
  * freq_time       (freq_time) float64 -4.63e-05 -4.628e-05 ... 4.628e-05
  * latitude        (latitude) float32 10.0 9.75 9.5 9.25 ... -9.5 -9.75 -10.0
  * freq_longitude  (freq_longitude) float64 -2.0 -1.997 -1.994 ... 1.994 1.997
  * time            (time) datetime64[ns] 1990-09-03 ... 1992-04-29T21:00:00
Data variables:
    TF_tcwv         (freq_time, latitude, freq_longitude) complex128 dask.array<chunksize=(4840, 1, 1440), meta=np.ndarray>

In [15]:
tcwv_Rossby_Filt = xrft.ifft(tcwvhat['TF_tcwv'],
                    dim=['freq_time','freq_longitude'], true_phase=False, true_amplitude=True)
    

/home/durandy/miniconda3/envs/croco/lib/python3.7/site-packages/xrft/xrft.py:556: FutureWarning: Default idft's behaviour (lag=None) changed! Default value of lag was zero (centered output coordinates) and is now set to transformed coordinate's attribute: 'direct_lag'.
  warnings.warn(msg, FutureWarning)


In [16]:
tcwv_Rossby_Filt['time'] = tcwvhat_Rossby_Filt.time

In [17]:
p = tcwv_Rossby_Filt.real.sel(time = "1991-06-06").compute()

In [18]:
p

<xarray.DataArray 'rechunk-merge-16bc0951dd63cc8fa3e2645746f60512' (time: 8, latitude: 81, longitude: 1440)>
array([[[-1.40181206e+00, -7.34735682e-01, -5.01464933e-01, ...,
         -1.62510624e+00, -1.93003052e+00, -1.80177305e+00],
        [-1.11658231e+00, -6.50289004e-01, -4.42110755e-01, ...,
         -1.33387769e+00, -1.49757649e+00, -1.43741758e+00],
        [-5.17307439e-01, -5.39065101e-01, -5.44888874e-01, ...,
         -8.62477000e-01, -6.68409579e-01, -5.94252126e-01],
        ...,
        [-1.89084345e+00, -1.58676609e+00, -1.19459438e+00, ...,
         -2.71259697e+00, -2.50086283e+00, -2.19412418e+00],
        [-3.09243691e+00, -2.92030309e+00, -2.75839666e+00, ...,
         -3.59488524e+00, -3.40306254e+00, -3.24889092e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 9.81995711e-01,  1.12361131e+00,  6.28580482e-01, ...,
         -1.84579782e-01,  2.21213679e-02,  3.96616921e-01],
        [ 8.45126055e-01,  1.10039927e+00,  8.71535838e-01, ...,
         -3.73856438e-01, -1.64687692e-01,  1.18388251e-01],
        [ 6.68708147e-01,  7.97433653e-01,  5.01127956e-01, ...,
         -6.85883975e-01, -2.44019960e-01,  2.28307141e-03],
...
        [-3.74511668e+00, -3.55809737e+00, -3.41212869e+00, ...,
         -4.02084704e+00, -3.89942236e+00, -3.79597049e+00],
        [-3.74416534e+00, -3.57993211e+00, -3.49239213e+00, ...,
         -3.89038259e+00, -3.75167622e+00, -3.70316026e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.21459066e+00,  3.68794164e+00,  4.61022240e+00, ...,
          2.62481771e+00,  2.83285415e+00,  2.98353016e+00],
        [ 3.50824335e+00,  4.40764095e+00,  5.28888726e+00, ...,
          1.98227054e+00,  2.39348292e+00,  2.73862464e+00],
        [ 2.94535746e+00,  3.74464933e+00,  4.45537264e+00, ...,
          1.16701553e+00,  1.72660670e+00,  2.26548683e+00],
        ...,
        [-4.02723936e+00, -3.70707366e+00, -3.33228777e+00, ...,
         -4.38770738e+00, -4.35784253e+00, -4.22285548e+00],
        [-3.90564626e+00, -3.59463213e+00, -3.31539393e+00, ...,
         -4.33979005e+00, -4.25551924e+00, -4.15007847e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]])
Coordinates:
  * latitude   (latitude) float32 10.0 9.75 9.5 9.25 ... -9.25 -9.5 -9.75 -10.0
  * time       (time) datetime64[ns] 1991-06-06 ... 1991-06-06T21:00:00
  * longitude  (longitude) float64 -180.0 -179.8 -179.5 ... 179.3 179.5 179.8

In [24]:

p.isel(time = 1).plot(size = ((10,8)))

TypeError: can't multiply sequence by non-int of type 'float'